In [ ]:
import sys
import spotipy
import yaml
import spotipy.util as util
import requests
from bs4 import BeautifulSoup

def load_config():
    '''Load Spotify credentials from yaml file.'''
    global user_config
    stream = open('config.yaml')
    user_config = yaml.load(stream)

def get_artists():
    '''Return list of artist names from the SXSW website'''
    html = requests.get('https://schedule.sxsw.com/2019/artists/genre/Metal').text
    bs=BeautifulSoup(html, 'lxml')
    links=bs.find_all('h4')
    artists=[]
    for link in links:
        artists.append(link.string)
    return artists

def get_artist_ids(artists):
    '''Using Artist list, return list of artist IDs from Spotify'''
    a = artists
    artist_ids = []
    for i in range(len(a)-1):
        artist_info = sp.search(q='artist:' + a[i], type='artist')
        if artist_info['artists']['total']:
            artist_id=artist_info['artists']['items'][0]['id']
            artist_ids.append(artist_id)
    return artist_ids  

def get_top_tracks(artist_ids):
    '''Using Artist ID list, return list of track IDs from Spotify'''
    a = artist_ids
    track_ids = []
    for i in range(len(a)-1):
        top_tracks = sp.artist_top_tracks(a[i])
        for track in top_tracks['tracks'][:1]:
            track_ids.append(track['id'])
    return(track_ids)

def add_to_playlist(track_ids):
    '''Using track IDs, and Spotify credentials, add tracks to playlist'''
    sp.user_playlist_add_tracks(user=user_config['username'], playlist_id=user_config['playlist_id'], tracks=track_ids)


def main():
    global sp
    global user_config
    load_config()
    token = util.prompt_for_user_token(user_config['username'], scope='playlist-modify-private,playlist-modify-public', client_id=user_config['client_id'], client_secret=user_config['client_secret'], redirect_uri=user_config['redirect_uri'])
    if token:
        sp = spotipy.Spotify(auth=token)
        artists = get_artists()
        artist_ids = get_artist_ids(artists)
        top_tracks = get_top_tracks(artist_ids)
        add_to_playlist(top_tracks)
    else:
        print ("Can't get token for", user_config['username'])
        
if __name__ == "__main__":
    main()